### Dummy code

In [1]:
import os
import cv2
import math
import time
import tarfile
import numbers
import threading
import queue as Queue
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url
#from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, DataLoader, Dataset
from torchsummary import summary

In [ ]:
import mxnet as mx
from mxnet import recordio

#### Loss Function checking

In [ ]:
k = nn.Parameter(torch.FloatTensor(3, 5))
k

In [ ]:
a = torch.Tensor([[1, 2, 3], [3, -4, -2]])
b = torch.Tensor([[1, 2, 3, 4], [3, 4, 1, 0], [5, 6, 1, 1]])

In [ ]:
a.shape, b.shape

In [ ]:
#a = torch.FloatTensor([2, 3])
#a.dtype

In [ ]:
c = torch.mm(a, b)
c.shape

In [ ]:
c

In [ ]:
c.size()

In [ ]:
e = c.clamp(-1, 1)

In [ ]:
e

In [ ]:
dirty = e > 0
dirty

In [ ]:
dirty = dirty.float()
dirty

In [ ]:
index = torch.where(c != -1)[0]
index

In [ ]:
index.size(0)

In [ ]:
index.size()[0]

In [ ]:
torch.zeros(index.size()[0], c.size()[1])

In [ ]:
def abc(a: int, logits: torch.IntTensor):
    print(a)
    print(logits)

In [ ]:
abc(5.2, c)

In [ ]:
k = [[1, 2, 3], [2, 3, 1]]
abc(5, k)

In [ ]:
k = [[1, 2, 3], [2, 3, 1]]
abc(5.2, 4)

In [ ]:
c.view(-1)

In [ ]:
#----

In [ ]:
logits = torch.Tensor([[1, 0, 0, 0], [0, 1, 1, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0]]) #torch.Tensor([10, 2, 3, 5])
labels = torch.Tensor([[1, 0, 0, 0], [0, 1, 1, 0], [0, 1, 0, 1], [0, 0, 0, 1]])

index = torch.where(labels != -1)[0]
index

In [ ]:
print(logits.dtype)
print(labels.dtype)
print(index.dtype)

In [ ]:
labels[index]

In [ ]:
l = labels[index].view(-1)
l

In [ ]:
l = l.int()
l

In [ ]:
t = logits[index, index]

In [ ]:
t

#### Loss Function 1

In [ ]:
class CosFace(torch.nn.Module):
    def __init__(self, s=64.0, m=0.40):
        super(CosFace, self).__init__()
        self.s = s
        self.m = m

    def forward(self, logits, labels):
        index = torch.where(labels != -1)[0]
        target_logit = logits[index, labels[index].view(-1)]
        final_target_logit = target_logit - self.m
        logits[index, labels[index].view(-1)] = final_target_logit
        logits = logits * self.s
        return logits

In [ ]:
loss = CosFace()

In [ ]:
logits = torch.randn(3, 5)
labels = torch.empty(3, dtype=torch.long).random_(5)

In [ ]:
logits.dtype, labels.dtype

In [ ]:
output = loss(logits, labels) #input: logits, target: labels

In [ ]:
print(logits.shape, labels.shape, output.shape)

In [ ]:
print('input: ', logits, '\n target: ', labels, '\n output: ', output)

In [ ]:
index = torch.where(labels != -1)[0]
index

In [ ]:
labels[index].view(-1)

In [ ]:
target_logit = logits[index, labels[index].view(-1)]
target_logit

In [ ]:
final_target_logit = target_logit - 1
final_target_logit

In [ ]:
logits[index, labels[index].view(-1)] = final_target_logit
logits

In [ ]:
logits = logits * 100
logits

#### Loss Function 2

In [ ]:
class CosFace(nn.Module):
    def __init__(self, in_features, out_features, s=64.0, m=0.35):
        super(CosFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        
        self.kernel = nn.Parameter(torch.FloatTensor(in_features, out_features))
        nn.init.normal_(self.kernel, std=0.01)

    def forward(self, logits, labels):
        #logits = l2_norm(logits, axis=1)
        #kernel_norm = l2_norm(self.kernel, axis=0)
        cos_theta = torch.mm(logits, kernel)#_norm)
        cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
        index = torch.where(labels != -1)[0]
        m_hot = torch.zeros(index.size()[0], cos_theta.size()[1])
        m_hot.scatter_(1, labels[index, None], self.m)
        cos_theta[index] -= m_hot
        ret = cos_theta * self.s
        return ret

In [ ]:
torch.linalg.norm(logits, dim=1, ord = 2, keepdim=True)

In [ ]:
F.normalize(logits, p=2.0, dim=0)

In [20]:
logits = torch.randn(128, 100)
labels = torch.empty(128, dtype=torch.long).random_(5)

In [21]:
logits.dtype, labels.dtype

(torch.float32, torch.int64)

In [22]:
print(logits.shape, labels.shape)#, output.shape)

torch.Size([128, 100]) torch.Size([128])


In [ ]:
print('logits: ', logits, '\n labels: ', labels)#, '\n output: ', output)

In [23]:
in_features = 100
out_features = 500
s = 10
m = 5
        
kernel = nn.Parameter(torch.FloatTensor(in_features, out_features))

In [24]:
kernel.shape

torch.Size([100, 500])

In [25]:
cos_theta = torch.mm(logits, kernel)
cos_theta, cos_theta.shape

(tensor([[ 0.0000e+00,  0.0000e+00, -6.7965e-20,  ...,  0.0000e+00,
          -6.3199e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -8.2432e-20,  ...,  0.0000e+00,
           3.1809e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.5335e-19,  ...,  0.0000e+00,
           8.5059e-43,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00, -1.2393e-20,  ...,  0.0000e+00,
          -6.0676e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -7.1846e-20,  ...,  0.0000e+00,
          -3.0688e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  6.8737e-20,  ...,  0.0000e+00,
           3.4332e-43,  0.0000e+00]], grad_fn=<MmBackward0>),
 torch.Size([128, 500]))

In [26]:
cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
cos_theta, cos_theta.shape

(tensor([[ 0.0000e+00,  0.0000e+00, -6.7965e-20,  ...,  0.0000e+00,
          -6.3199e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -8.2432e-20,  ...,  0.0000e+00,
           3.1809e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.5335e-19,  ...,  0.0000e+00,
           8.5059e-43,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00, -1.2393e-20,  ...,  0.0000e+00,
          -6.0676e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -7.1846e-20,  ...,  0.0000e+00,
          -3.0688e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  6.8737e-20,  ...,  0.0000e+00,
           3.4332e-43,  0.0000e+00]], grad_fn=<ClampBackward1>),
 torch.Size([128, 500]))

In [27]:
index = torch.where(labels != -1)[0]
index, index.shape

(tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127]),
 torch.Size([128]))

In [28]:
index.shape[0], cos_theta.size()[1]

(128, 500)

In [29]:
m_hot = torch.zeros(index.size()[0], cos_theta.size()[1])
m_hot.shape, m_hot

(torch.Size([128, 500]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [30]:
labels[index, None]

tensor([[0],
        [1],
        [3],
        [2],
        [2],
        [0],
        [4],
        [0],
        [4],
        [3],
        [2],
        [3],
        [4],
        [1],
        [0],
        [0],
        [2],
        [4],
        [2],
        [0],
        [1],
        [0],
        [2],
        [4],
        [3],
        [2],
        [4],
        [2],
        [0],
        [4],
        [4],
        [3],
        [4],
        [2],
        [4],
        [2],
        [1],
        [4],
        [2],
        [1],
        [1],
        [3],
        [0],
        [4],
        [4],
        [1],
        [3],
        [0],
        [2],
        [2],
        [4],
        [0],
        [1],
        [3],
        [0],
        [1],
        [2],
        [0],
        [3],
        [4],
        [4],
        [0],
        [1],
        [2],
        [2],
        [2],
        [0],
        [1],
        [3],
        [3],
        [2],
        [0],
        [1],
        [2],
        [3],
        [4],
        [1],

In [31]:
m_hot.scatter_(1, labels[index, None], m)

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [37]:
m_hot

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [38]:
cos_theta[index] -= m_hot
cos_theta

tensor([[-1.0000e+01,  0.0000e+00, -6.7965e-20,  ...,  0.0000e+00,
         -6.3199e-43,  0.0000e+00],
        [ 0.0000e+00, -1.0000e+01, -8.2432e-20,  ...,  0.0000e+00,
          3.1809e-43,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -1.5335e-19,  ...,  0.0000e+00,
          8.5059e-43,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00, -1.2393e-20,  ...,  0.0000e+00,
         -6.0676e-43,  0.0000e+00],
        [ 0.0000e+00, -1.0000e+01, -7.1846e-20,  ...,  0.0000e+00,
         -3.0688e-43,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  6.8737e-20,  ...,  0.0000e+00,
          3.4332e-43,  0.0000e+00]], grad_fn=<IndexPutBackward0>)

In [35]:
a = cos_theta - m_hot

In [36]:
a, a.shape

(tensor([[-1.0000e+01,  0.0000e+00, -6.7965e-20,  ...,  0.0000e+00,
          -6.3199e-43,  0.0000e+00],
         [ 0.0000e+00, -1.0000e+01, -8.2432e-20,  ...,  0.0000e+00,
           3.1809e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.5335e-19,  ...,  0.0000e+00,
           8.5059e-43,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00, -1.2393e-20,  ...,  0.0000e+00,
          -6.0676e-43,  0.0000e+00],
         [ 0.0000e+00, -1.0000e+01, -7.1846e-20,  ...,  0.0000e+00,
          -3.0688e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  6.8737e-20,  ...,  0.0000e+00,
           3.4332e-43,  0.0000e+00]], grad_fn=<SubBackward0>),
 torch.Size([128, 500]))

In [33]:
ret = cos_theta * s
ret

tensor([[-5.0000e+01,  0.0000e+00, -6.7965e-19,  ...,  0.0000e+00,
         -6.3199e-42,  0.0000e+00],
        [ 0.0000e+00, -5.0000e+01, -8.2432e-19,  ...,  0.0000e+00,
          3.1809e-42,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -1.5335e-18,  ...,  0.0000e+00,
          8.5059e-42,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00, -1.2393e-19,  ...,  0.0000e+00,
         -6.0676e-42,  0.0000e+00],
        [ 0.0000e+00, -5.0000e+01, -7.1846e-19,  ...,  0.0000e+00,
         -3.0688e-42,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  6.8737e-19,  ...,  0.0000e+00,
          3.4332e-42,  0.0000e+00]], grad_fn=<MulBackward0>)

In [34]:
ret.shape

torch.Size([128, 500])

In [ ]:
loss = CosFace(5, 5)

In [ ]:
output = loss(logits, labels) #input: logits, target: labels
output.shape, output

#### Dataset preprocessing checking

##### For Windows system

In [ ]:
# path_imgidx = 'D:/Face/faces_emore/train.idx'
# path_imgrec = 'D:/Face/faces_emore/train.rec'
# imgrec = recordio.MXIndexedRecordIO(path_imgidx, path_imgrec, 'r')

# len(imgrec.keys)

##### For Ubuntu system

In [ ]:
path_imgidx = 'faces_emore/train.idx'
path_imgrec = 'faces_emore/train.rec'
imgrec = recordio.MXIndexedRecordIO(path_imgidx, path_imgrec, 'r')

len(imgrec.keys)

In [ ]:
imgrec.keys

In [ ]:
list(imgrec.keys)

In [ ]:
np.array(list(imgrec.keys))

In [ ]:
len(np.array(list(imgrec.keys)))

In [ ]:
img_list = []

#%% 1 ~ 5908396, or 0~5908395, #85742 identities

for i in range(5908395):
    header, s = recordio.unpack(imgrec.read_idx(i+1))
#     img = mx.image.imdecode(s).asnumpy() 
#     plt.imshow(img)
#     print(img.shape)
#     plt.title('id=' + str(i) + ', label=' + str(header.label))
#     plt.pause(0.1)
    img_list.append(header.label)

In [ ]:
len(img_list)

In [ ]:
# img_list = np.array(img_list)
# print(np.unique(img_list))

In [ ]:
_img_list = img_list[0:5822653]
__img_list = np.array(_img_list)
print(len(__img_list))
print(len(np.unique(__img_list)))

In [ ]:
# res_list = []

# for item in _img_list:
#     if item not in res_list:
#         res_list.append(item)

# print("Unique elements of the list using append():")
# print(len(res_list))

# #for item in res_list: 
# #    print(item)

In [ ]:
5908395 - 5822653

In [ ]:
img_list[5822653]

In [ ]:
img_list[5822654]

In [ ]:
idx1 = 1000 #5908395
header1, s1 = recordio.unpack(imgrec.read_idx(idx1))

img1 = mx.image.imdecode(s1).asnumpy()
plt.imshow(img1)
plt.title('id=' + str(idx1) + ', label=' + str(header1.label))

In [ ]:
idx = 5822656
header, s = recordio.unpack(imgrec.read_idx(idx))

#img = mx.image.imdecode(s).asnumpy()
#plt.imshow(img)
plt.title('id=' + str(idx) + ', label=' + str(header.label))

In [ ]:
header.flag

In [ ]:
header.label

In [ ]:
header0 = (int(header.label[0]), int(header.label[1]))
header0

In [ ]:
np.array(range(1, int(header.label[0])))

In [ ]:
len(np.array(range(1, int(header.label[0]))))

In [ ]:
len(np.array(range(1, int(header.label[1]))))

In [ ]:
class Sub:
    def __init__(self, name):
        super(Sub, self).__init__() #Another way to fix  this
        self.name=name
        

a = Sub("Harry")
a.name

In [ ]:
isinstance([1, 2], numbers.Number)

In [ ]:
a = torch.tensor(65.5, dtype=torch.long)
a

#### Training checking

In [ ]:
temp = torch.zeros(85742)
#temp[idx] = 1
#temp

In [ ]:
temp_labels = torch.Tensor([5, 5, 1, 0]) #batchsize=4
print(temp_labels)
print(temp_labels[0])

temp = torch.zeros(4, 6) #no.of classes=6 : [0-5]
#temp[int(temp_labels[0])] = 1
print(temp.shape)

In [ ]:
c = 0
for i in temp_labels:
    #print(i)
    #print(c)
    temp[c][int(i)] = 1
    c+=1

In [ ]:
temp

In [ ]:
a = torch.Tensor([[1, 0, 2], [1, 1, 2]]).to('cuda') #torch.ones(4)
a[0].shape, a.shape, a

In [ ]:
b.is_cpu

In [ ]:
b = torch.sum(a)
b

In [ ]:
c = F.softmax(a, dim=0)
c

In [ ]:
# d = F.softmax(d, dim=0)
# d

In [ ]:
torch.sum(c, dim=0)

In [ ]:
actual_labels = torch.zeros(256, 85742)

c = 0
for i in labels:
    actual_labels[c][int(i)] = 1
    c+=1

In [ ]:
actual_labels.shape

In [ ]:
actual_labels[5][4966]

In [ ]:
actual_labels[5]

#### Check by using various loss functions of the library

In [ ]:
# for xb, yb in val_dl:
#     print(xb.shape, yb.shape)
#     temp_preds = model(xb)
#     print(temp_preds.shape)
#     temp_loss = temp_loss_func(temp_preds, yb)
#     print(temp_loss)
#     break

In [ ]:
temp_loss_func = nn.CrossEntropyLoss() #F.cross_entropy #F.binary_cross_entropy #

In [ ]:
a = torch.Tensor([[0, 0, 0]]) #torch.zeros(256, 85742) #
b = torch.Tensor([2]) #torch.zeros(256) #
b = b.to(torch.long)
print(a.shape, b.shape, a.dtype, b.dtype)

In [ ]:
c = temp_loss_func(a, b)
c.shape, c.item()

In [ ]:
input = torch.randn(3, 5)
target = torch.empty(3, dtype=torch.long).random_(2)
output = temp_loss_func(input, target)

In [ ]:
input.shape, target.shape, output.shape

In [ ]:
input, target, output

#### Training Accuracy checking

In [ ]:
a = torch.Tensor([[1, 2, 33, -4, -2, 1], [5, 55, 1, -4, 0, 56]])
b = torch.Tensor([2, 5])

c = accuracy(a, b)
print(c)

In [ ]:
_, preds = torch.max(a, dim=1)
preds

In [ ]:
d = torch.sum(preds == b)
d.item()

#### Printing results

In [ ]:
def fetch_pred_image(lst, idx):
    idxs = np.where(lst==idx)[0]
    
    path_imgidx = os.path.join(root_dir1, 'train.idx')
    path_imgrec = os.path.join(root_dir1, 'train.rec')
    imgrec = recordio.MXIndexedRecordIO(path_imgidx, path_imgrec, 'r')
    header, s = recordio.unpack(imgrec.read_idx(idxs[-1]))
    img = mx.image.imdecode(s).asnumpy()
    plt.imshow(img)
    plt.title('label = ' + str(header.label))
    plt.axis('off')
    
#     img, label = dataset[idxs[-1]]
#     img = torch.reshape(img, (112, 112, 3))
#     plt.imshow(img)
#     plt.title('label = ' + str(label.item()))
#     plt.show()

In [ ]:
fetch_pred_image(lst, 59062)

In [ ]:
def prepare_plot(image, origTarget, predTarget):
    plt.grid(False)
    
    id1 = 11
    id2 = 110
    id3 = 123
    
    img_fetch1 = fetch_pred_image(imgrec, lst, origTarget[id1])
    img_fetch2 = fetch_pred_image(imgrec, lst, origTarget[id2])
    img_fetch3 = fetch_pred_image(imgrec, lst, origTarget[id3])
    
    
    plt.subplot(3, 2, 1)
    plt.imshow(image[id1])
    plt.axis('off')
    plt.title('Predicted id: ' + str(predTarget[id1]))
    
    plt.subplot(3, 2, 2)
    plt.imshow(np.real(img_fetch1))
    plt.axis('off')
    plt.title('Actual id: ' + str(origTarget[id1]))
    
    
    plt.subplot(3, 2, 3)
    plt.imshow(image[id2])
    plt.axis('off')
    plt.title('Predicted id: ' + str(predTarget[id2]))
    
    plt.subplot(3, 2, 4)
    plt.imshow(np.real(img_fetch2))
    plt.axis('off')
    plt.title('Actual id: ' + str(origTarget[id2]))
    
    
    plt.subplot(3, 2, 5)
    plt.imshow(image[id3])
    plt.axis('off')
    plt.title('Predicted id: ' + str(predTarget[id3]))
    
    plt.subplot(3, 2, 6)
    plt.imshow(np.real(img_fetch3))
    plt.axis('off')
    plt.title('Actual id: ' + str(origTarget[id3]))
    
    plt.show()

In [ ]:
def prepare_plot(image, origTarget, predTarget, size=5):
    plt.rcParams["figure.figsize"] = (15*size/25, 15*size/5)
    rand = np.random.randint(0, 250, size)
    
    for i in range(size):
        img_fetch = fetch_pred_image(imgrec, lst, predTarget[rand[i]])
        plt.subplot(size, 2, 2*i+1)
        plt.imshow(image[rand[i]])
        plt.axis('off')
        plt.title('Actual id: ' + str(origTarget[rand[i]]))
        
        plt.subplot(size, 2, 2*i+2)
        plt.imshow(np.real(img_fetch))
        plt.axis('off')
        plt.title('Predicted id: ' + str(predTarget[rand[i]]))
    
    plt.show()